In [ ]:
# model setup


In [ ]:
import mysql.connector
from mysql.connector import Error

# Establishing database connection
def get_db_connection(host, database, user, password):
    try:
        connection = mysql.connector.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
        else:
            raise Exception("Failed to connect to database.")
    except Error as e:
        raise Exception("Failed to connect to database:{e}")
        return None

# Function to get the model's query based on the question
def get_query_from_model(question, model):
    # Assume `model` is an object with a `generate_query` method
    query = model.generate_query(question)
    return query

# Function to log the query based on user feedback
def insert_human_feedback(connection, question, query, is_right, error_description=None):
    try:
        cursor = connection.cursor()
        
        # SQL query to insert the data
        insert_query = """
        INSERT INTO queries_log (question, query, is_right, error_description)
        VALUES (%s, %s, %s, %s)
        """
        
        # Values to insert
        values = (question, query, is_right, error_description if error_description else None)
        
        # Execute the query
        cursor.execute(insert_query, values)
        connection.commit()
        print("Query log inserted successfully.")
    
    except Error as e:
        print(f"Error: {e}")
    
    finally:
        cursor.close()

# Function to capture user feedback and log data
def capture_feedback_and_log(question, db_connection, db_langchain):

    # Step 1: Get the query from the model
    query = getQueryFromLLM(question,db_langchain)
    print(f"Model generated query: {query}")

    # Step 2: Ask user if the output is correct
    is_right = input("Is the model output correct? (yes/no): ").strip().lower()

    # Step 3: If wrong, capture explanation
    error_description = None
    if is_right == 'no':
        error_description = input("Why is the model output wrong? Please provide explanation: ")

    # Step 4: Insert data into database
    insert_human_feedback(db_connection, question, query, is_right, error_description)

    print("Feedback logged successfully.")

# Main function to set up the interaction
def main():
    # Define your DB connection parameters
    host = 'localhost'
    database = 'atliq_tshirts'
    user = 'root'
    password = 'Atk%408522'

    # Get database connection
    db_connection = get_db_connection(host, database, user, password)
    

    if not db_connection:
        print("Failed to connect to the database. Exiting.")
        return

    db=connectDatabase(username='root', port='3306', host='localhost', password='Atk%408522', database='atliq_tshirts')

    while True:
        question = input("Enter your question: ")
        if question.lower() == 'exit':
            break

        capture_feedback_and_log(question, db_connection, db_langchain)

    # Close the database connection
    if db_connection:
        db_connection.close()
        print("MySQL connection closed.")

if __name__ == "__main__":
    main()
